In [ ]:
import cudf
import cugraph as cg

import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Networkx

Создадим простой граф при помощи networkx

In [ ]:
G = nx.Graph()

добавим 1 ноду к графу

In [ ]:
G.add_node(1)

In [ ]:
G.nodes

Можно также добавить несколько нод сразу

In [ ]:
G.add_nodes_from([2, 3])
G.nodes

Добавим также связи между узлами

In [ ]:
G.add_edges_from([(1, 2), (1, 3)])

In [ ]:
G.edges

## cuGraph

Давайте повторим это все в cugraph

cg.Graph() - ненаправленный граф, cg.DiGraph() - направленный граф

In [ ]:
G_gpu = cg.Graph()

add_node к сожалению нет, но есть add_nodes_from

In [ ]:
G_gpu.add_nodes_from([1])
G_gpu.nodes()

Нюанс, ноды перезаписываются, так как считается, что мы передаем все необходимые ноды

In [ ]:
G_gpu.add_nodes_from([2, 3])
G_gpu.nodes()

Чтобы добавить что-то новое придется заново инициализировать граф

Можно через pandas, numpy, cudf, dask-cudf

In [ ]:
edge_list = cudf.DataFrame({
    'source': [1, 1],
    'destination': [2, 3]
})

In [ ]:
G_gpu = cg.Graph()
G_gpu.from_cudf_edgelist(edge_list, source='source', destination='destination')

In [ ]:
G_gpu.nodes()

In [ ]:
G_gpu.edges()

Например, если захотим добавить что-то новое, то можно сделать так:

In [ ]:
appended_values = cudf.DataFrame({
    'source': [2],
    'destination': [3]
})
edge_list = edge_list.append(appended_values, ignore_index=True)

In [ ]:
edge_list

In [ ]:
G_gpu = cg.Graph()
G_gpu.from_cudf_edgelist(edge_list, source='source', destination='destination')
G_gpu.edges()

## Давайте загрузим известный датасет карате клуб и попробуем поанализировать его в разных фреймворках

## Networkx

https://raw.githubusercontent.com/rapidsai/notebooks/branch-0.8/cugraph/data/karate-data.csv

In [ ]:
edges = pd.read_csv('karate-data.csv', sep='\t', names=['source', 'destination'])

In [ ]:
edges

In [ ]:
G = nx.Graph()
G = nx.from_pandas_edgelist(edges, source='source', target='destination')

In [ ]:
nx.draw(G, with_labels=True)

Давайте поисследуем сам граф

Средняя степень узлов сети и общие характеритики

In [ ]:
def mean_nodes_degree(g):
    return 2 * len(g.edges) / len(g.nodes)

In [ ]:
def get_statistics(g):
    print(f'Number of nodes = {len(g.nodes)}')
    print(f'Number of edges = {len(g.edges)}')

In [ ]:
get_statistics(G)

In [ ]:
mean_nodes_degree(G)

In [ ]:
G.degree

Каждый узел важен для сети по разному и для определения важности есть меры центральности. Мы рассмотрим центральность по степени и центральность по посредничеству

In [ ]:
def get_centrality_by_degree(g):
    centrality = nx.algorithms.centrality.degree_centrality(g)
    return sorted(list(centrality.items()), key=lambda x: x[1], reverse=True)

In [ ]:
get_centrality_by_degree(G)

In [ ]:
def get_centrality_by_betweenness(g):
    centrality = nx.algorithms.centrality.betweenness_centrality(g)
    return sorted(list(centrality.items()), key=lambda x: x[1], reverse=True)

In [ ]:
get_centrality_by_betweenness(G)

PageRank

In [ ]:
nx.pagerank(G,alpha=0.9)

Пора сделать аналогичное в cuGraph

## CuGraph

In [ ]:
edges_gpu = cudf.read_csv('karate-data.csv', sep='\t', names=['source', 'destination'])

In [ ]:
edges_gpu

In [ ]:
G_gpu = cg.Graph()
G_gpu.from_cudf_edgelist(edges_gpu, source='source', destination='destination')

Посчитаем статистики, как и в networkx

In [ ]:
def mean_nodes_degree(g):
    return 2 * len(g.edges()) / len(g.nodes())

In [ ]:
def get_statistics(g):
    print(f'Number of nodes = {len(g.nodes())}')
    print(f'Number of edges = {len(g.edges())}')

In [ ]:
get_statistics(G_gpu)

In [ ]:
mean_nodes_degree(G_gpu)

Надо быть внимательным, так как cugraph считает, что связь между нодами дает не 1 степень, а 2 (in и out)

In [ ]:
G_gpu.degree()

Меры центральности

Центральность по степени не запрогали за нас, сделаем сами=)

In [ ]:
def get_centrality_by_degree(g):
    centrality = g.degree()
    centrality['degree_centrality'] = centrality['degree'] / (len(g.nodes())-1) / 2
    centrality = centrality.sort_values(by='degree_centrality', ascending=False)
    return centrality

In [ ]:
get_centrality_by_degree(G_gpu)

In [ ]:
def get_centrality_by_betweenness(g):
    centrality = cg.centrality.betweenness_centrality(g)
    centrality = centrality.sort_values(by='betweenness_centrality', ascending=False)
    return centrality

In [ ]:
get_centrality_by_betweenness(G_gpu)

PageRank

In [ ]:
cg.link_analysis.pagerank(G_gpu)

Много чего нет в cuGraph, но есть и преимущества)
Например, в Networkx не реализованы алгоритмы кластеризации (их мало), а многие из существующих есть в cuGraph

Метод louvain на основе меры модулярности

In [ ]:
def louvain_clusters(g):
    result = cg.community.louvain(g)
    modularity = result[1]
    clusters = result[0]
    return modularity, clusters

In [ ]:
modularity, clusters = louvain_clusters(G_gpu)

In [ ]:
modularity

In [ ]:
clusters = clusters.to_pandas()
clusters = clusters.sort_values(by='vertex')

In [ ]:
clusters = pd.DataFrame(G.nodes).merge(clusters, left_on=0, right_on='vertex')

Отрисуем граф при помощи Networkx

In [ ]:
nx.draw(G,node_color=clusters['partition'],node_size=200)

## Multi-GPU

Это нам уже знакомо. Используем dask-cudf и радуемся тому, что теперь все можно делать на нескольких GPU

In [ ]:
import subprocess
from dask_cuda import LocalCUDACluster
from dask.distributed import Client, progress
import dask_cudf
import dask.dataframe as dd

In [ ]:
cmd = "hostname --all-ip-addresses"

process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
IPADDR = str(output.decode()).split()[0]

cluster = LocalCUDACluster(ip=IPADDR)
client = Client(cluster)
client

In [ ]:
edges_gpu = dask_cudf.read_csv('karate-data.csv', sep='\t', names=['source', 'destination'])

In [ ]:
G_gpu = cg.Graph()
G_gpu.from_cudf_edgelist(edges_gpu, source='source', destination='destination')

In [ ]:
type(G_gpu.nodes())

In [ ]:
get_centrality_by_betweenness(G_gpu)

Ниже представлен алгоритм Гирвана-Ньюмена по поиску сообществ. Его идея:

    1) Находим такое ребро, через которое проходит наибольшее количество кратчайших путей
    2) Удаляем данное ребро из графа
    3) После удаления ребра проверяем, не появились ли у нас изолированные сообщества
    4) Повторяем алгоритм до получения нужного количества сообществ

# Girvan-Newman (NetworkX)

In [ ]:
def most_central_edge(G):
    centrality = nx.algorithms.centrality.edge_betweenness_centrality(G)
    return max(centrality, key=centrality.get)

In [ ]:
def get_n_cluster(G, n_clusters):
    G_new = G.copy()
    while len(list(nx.connected_components(G_new))) < n_clusters:
        edge_to_remove = most_central_edge(G_new)
        G_new.remove_edge(edge_to_remove[0], edge_to_remove[1])
    return list(nx.connected_components(G_new))

In [ ]:
def get_color_map(G, connected_components):
    b = {}
    for i, values in enumerate(connected_components):
        for value in values:
            b[value] = i
    color_map = []
    for node in G.nodes():
        color_map.append(b[node])
    return color_map

In [ ]:
clusters = get_n_cluster(G, 4)
nx.draw(G,node_size=200, with_labels=True, node_color=get_color_map(G, clusters))

# Домашнее задание

В NetworkX есть алгоритм определения центральности через собственные вектора nx.algorithms.centrality.eigenvector_centrality(G). Сейчас мы рассмотрим реализацию данного алгоритма.
Заданием будет следующее:

    1) Переписать реализацию под GPU (cupy, numba, cudf, cugraph...)
    2) Для графа карате клуба вычислить degree centrality, betweenness centrality, eigenvector centrality и pagerank.
    3) Сравнить 4 характеристики между собой, на сколько они похожи \ отличаются и в чем

In [ ]:
nx.algorithms.centrality.eigenvector_centrality(G)

In [ ]:
def eigen_centrality(G):
    #переведем в матрицу numpy
    M = nx.to_numpy_matrix(G)
    #получим собственные вектора и собственные значения
    eigenvalue, eigenvector = np.linalg.eig(M)
    #берем вектор, соответсвующий наибольшему собственному значению
    largest = np.array(eigenvector[:, 0].flatten().real)[0]
    #нормируем
    norm = np.sign(largest.sum()) * np.linalg.norm(largest)
    eigen = largest / norm
    return dict(zip(G, eigen))

In [ ]:
eigen_centrality(G)

Немного помощи с ДЗ

In [ ]:
#если не добавить веса, то будет все ломаться
edges_gpu['weights'] = 1

In [ ]:
#результаты будут отличаться от networkx в том случае, если граф взять ненаправленным. Тут возьмите направленный!
#влияет на матрицу смежности, собственные вектора получите, но они будут другими
G_gpu = cg.DiGraph()
G_gpu.from_cudf_edgelist(edges_gpu, source='source', destination='destination', edge_attr='weights')

P.S. будьте внимательны с тем, где находится наибольшее собственное значение